In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


In [11]:
df= pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Housing.csv')
df.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [12]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [13]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

In [14]:
ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False,
).set_output(transform='pandas')


In [15]:
X = df.drop('price', axis=1)
y = df['price']

X_transformed = ct.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=24)



### Elastic Net Regression

In [16]:
# Elastic Net Regression
alphas = np.linspace(0.0001, 10, 20)  
l1 = np.linspace(0.0001, 1, 10)
scores = []

for a in alphas:
    for l in l1:
        er = ElasticNet(alpha=a, l1_ratio=l)
        er.fit(X_train, y_train)
        y_pred = er.predict(X_test)
        scores.append({
            'alpha': a,
            'l1_ratio': l,
            'score': r2_score(y_test, y_pred)
        })

df_scores = pd.DataFrame(scores)
df_scores.sort_values('score', ascending=False, inplace=True)

print(f'Best parameters: \nalpha: {df_scores.alpha.iloc[0]} \nl1_ratio: {df_scores.l1_ratio.iloc[0]} \nscore: {df_scores.score.iloc[0]}\n\n')

print(df_scores)

Best parameters: 
alpha: 0.0001 
l1_ratio: 1.0 
score: 0.6246856181760903


         alpha  l1_ratio     score
9     0.000100    1.0000  0.624686
8     0.000100    0.8889  0.624684
7     0.000100    0.7778  0.624682
19    0.526411    1.0000  0.624680
6     0.000100    0.6667  0.624680
..         ...       ...       ...
181   9.473689    0.1112  0.266474
191  10.000000    0.1112  0.263793
170   8.947379    0.0001  0.263488
180   9.473689    0.0001  0.260782
190  10.000000    0.0001  0.258328

[200 rows x 3 columns]


In [17]:
elastic = ElasticNet(alpha=df_scores.alpha.iloc[0], l1_ratio=df_scores.l1_ratio.iloc[0])
elastic.fit(X_train, y_train)


for col, cof  in zip(list(X_test.columns), list(elastic.coef_)):
    print(f'{col}: {cof}')


lotsize: 3.6079332016655394
bedrooms: 1903.7838032104125
bathrms: 14258.360481576008
stories: 6508.231124462992
garagepl: 5155.034104572056
driveway_yes: 7379.70949281798
recroom_yes: 3605.8712566757386
fullbase_yes: 7306.3302904120665
gashw_yes: 10887.71250387912
airco_yes: 12316.443822657257
prefarea_yes: 7660.796299355298
